In [1]:
import re
import os
import csv

In [2]:
#dir_path = os.path.dirname(os.path.realpath(__file__))
dir_path = os.path.abspath('.')
print(dir_path)
debug = True

/home/yihan/framework/data


In [4]:
#os.listdir("cmu-wafer/normal")

In [7]:
def parse_filename(filename: str):
    return tuple(re.split("_|\.", filename))

# example:
if debug:
    print(parse_filename('1551_05.12'))

('1551', '05', '12')


In [8]:
# How to use
#list(map(parse_filename, _2))

In [9]:
# 반복문 사용법

for filename in os.listdir("cmu-wafer/normal"):
    with open(os.path.join("cmu-wafer/normal", filename)) as f:
        print(f.readlines())
    break

FileNotFoundError: [Errno 2] No such file or directory: 'cmu-wafer/normal'

In [10]:
# 단일 파일 읽기 함수 - normal
def read_data(datatype: str, filename: str):
    
    assert datatype == "normal" or datatype == "abnormal"
    label = True if datatype == "normal" else False
    
    with open(os.path.join("cmu-wafer", datatype, filename)) as f:
        r = csv.reader(f, delimiter="\t", 
                       skipinitialspace=True, 
                       quoting=csv.QUOTE_NONE)
        # 데이터 컬럼은 리스트 1번째 아이템이므로(0번째는 index), dl 
        data = [row[1] for row in r]

    return (data, label)

In [59]:
# read_data("normal", "1551_05.11")

### How to read

* path : `$PATH/data/cmu-wafer/`
    * normal item path : `.../normal/(filename)`
    * abnormal item path : `.../abnormal/(filename)`
    * do not read files with an `.ann` extension

In [10]:
data = []
labels = []

for filename in os.listdir("cmu-wafer/normal"):
    if not ".ann" in filename:
        #print(filename)
        d, l = read_data("normal", filename)
        data.append(d)
        labels.append(l)

for filename in os.listdir("cmu-wafer/abnormal"):
    if not ".ann" in filename and not ".box" in filename:
        #print(filename)
        d, l = read_data("abnormal", filename)
        data.append(d)
        labels.append(l)


In [12]:
os.listdir("cmu-wafer/abnormal/")

['1549_01.11',
 '1549_01.12',
 '1549_01.15',
 '1549_01.6',
 '1549_01.7',
 '1549_01.8',
 '1549_01.ann',
 '1549_02.11',
 '1549_02.12',
 '1549_02.15',
 '1549_02.6',
 '1549_02.7',
 '1549_02.8',
 '1549_02.ann',
 '1549_04.11',
 '1549_04.12',
 '1549_04.15',
 '1549_04.6',
 '1549_04.7',
 '1549_04.8',
 '1549_04.ann',
 '1549_06.11',
 '1549_06.12',
 '1549_06.15',
 '1549_06.6',
 '1549_06.7',
 '1549_06.8',
 '1549_06.ann',
 '1549_07.11',
 '1549_07.12',
 '1549_07.15',
 '1549_07.6',
 '1549_07.7',
 '1549_07.8',
 '1549_07.ann',
 '1549_08.11',
 '1549_08.12',
 '1549_08.15',
 '1549_08.6',
 '1549_08.7',
 '1549_08.8',
 '1549_08.ann',
 '1559_12.11',
 '1559_12.12',
 '1559_12.15',
 '1559_12.6',
 '1559_12.7',
 '1559_12.8',
 '1559_12.ann',
 '1564_02.11',
 '1564_02.12',
 '1564_02.15',
 '1564_02.6',
 '1564_02.7',
 '1564_02.8',
 '1564_02.ann',
 '1564_02.box',
 '1565_21.11',
 '1565_21.12',
 '1565_21.15',
 '1565_21.6',
 '1565_21.7',
 '1565_21.8',
 '1565_21.ann',
 '1565_21.box',
 '1571_01.11',
 '1571_01.12',
 '1571_01.1

In [15]:
len(data) / 6

1194.0

In [49]:
def _read_cmu_wafer():
    def dtoi(i: int):
        # map desc to index
        # 6, 7, 8, 11, 12, 15를 리스트로
        return ['6', '7', '8', '11', '12', '15'].index(i)
    
    def parse_filename(filename: str):
        return tuple(re.split("\.", filename))

    def read_data(datatype: str, filename: str):
        assert datatype == "normal" or datatype == "abnormal"
        label = True if datatype == "normal" else False
        
        with open(os.path.join("cmu-wafer", datatype, filename)) as f:
            r = csv.reader(f, delimiter="\t", 
                           skipinitialspace=True, 
                           quoting=csv.QUOTE_NONE)
            data = [row[1] for row in r]
        return (data, label) 

    data = dict()
    labels = dict()

    for filename in os.listdir("cmu-wafer/normal"):
        if not ".ann" in filename:
            run_wafer, desc = parse_filename(filename)
            if not run_wafer in data.keys():
                data[run_wafer] = [None] * 6
                
            d, l = read_data("normal", filename)
            data[run_wafer][dtoi(desc)] = d
            labels[run_wafer] = l

    for filename in os.listdir("cmu-wafer/abnormal"):
        if not ".ann" in filename and not ".box" in filename:
            run_wafer, desc = parse_filename(filename)
            if not run_wafer in data.keys():
                data[run_wafer] = [None] * 6
            
            d, l = read_data("abnormal", filename)
            data[run_wafer][dtoi(desc)] = d
            labels[run_wafer] = l

    return data, labels

In [51]:
data, labels = _read_cmu_wafer()

In [11]:
import numpy as np
np.array(data.values())[0]

NameError: name 'data' is not defined

In [33]:
from ds import Data, Pair

def _read_csv(filename, basedir="."):
    filedir = os.path.join(basedir, filename)
    with open(filedir, 'r') as csv_f:
        data = list(csv.reader(csv_f))
    # handles the data that have null column
    for i, row in enumerate(data):
        for j, item in enumerate(row):
            data[i][j] = '0.0' if item == '' else item
    return data


In [39]:
def _read_secom_preprocessed():
    # unsplitted data
    # 1568 -> 1199 + 368
    working_dir = os.path.join(dir_path, "uci-secom-preprocessed")
    
    data = Data(None, None, None, None)
    for filename in os.listdir(working_dir):
        if filename == "train.labels.csv":
            data.train.labels = np.asarray(_read_csv(filename, working_dir)).astype(int).flatten()
        if filename == "test.labels.csv":
            data.test.labels = np.asarray(_read_csv(filename, working_dir)).astype(int).flatten()
        if filename == "train_knnImpute.csv":
            data.train.data = np.asarray(_read_csv(filename, working_dir)).astype(float)
        if filename == "test_knnimpute.csv":
            data.test.data = np.asarray(_read_csv(filename, working_dir)).astype(float)

    return data

In [40]:
d = _read_secom_preprocessed()


In [21]:
working_dir = os.path.join(dir_path, "uci-secom-preprocessed")
os.listdir(working_dir)

['test.labels.csv',
 'test_knnImpute.csv',
 'train_knnImpute.csv',
 'train_knnImpute_recover.csv']

In [41]:
d.train.labels.shape

(1096,)